<img src='https://repository-images.githubusercontent.com/121802384/c355bb80-7d42-11e9-9e0e-4729609f9fbc' alt='WRF-Hydro Logo' width="15%"/>

# Lesson S2- Regridding NLDAS forcing data to the WRF-Hydro domain

## Overview
In this lesson, we cover regridding NLDAS2 forcing data to a WRF-Hydro domain using the Python-based WRF-Hydro Forcing Engine

## Required software and datasets

### Meteorological Forcing Engine (MFE) software
The WRF-Hydro Meteorological Forcing Engine (MFE) software can be downloaded from https://github.com/NCAR/WrfHydroForcing

### Data sources
For this training, data has already been included for the simulation time period. 

**NOTE: These data are for instructional purposes only and do not include forcing data used for model spinup**

If you would like to acquire other data, NLDAS2 forcing data can be retrieved from the NASA data server at https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_FORA0125_H.002/. 

See the following site for information on how to access data:
https://disc.gsfc.nasa.gov/data-access 

See the following instructions for how to download data files using wget:
https://disc.gsfc.nasa.gov/data-access#mac_linux_wget

**Step 1: Extract the NLDAS2 forcing data provided with this tutorial**

In [5]:
%%bash
cd ~/wrf-hydro-training/example_case/regridding
tar -xf nldas_mfe_forcing.tar.gz

**Step 2: View the contents of the input_files directory**

In [6]:
%%bash
ls ~/wrf-hydro-training/example_case/regridding/NLDAS/input_files | head -10

NLDAS_FORA0125_H.A20170101.0000.002.nc
NLDAS_FORA0125_H.A20170101.0100.002.nc
NLDAS_FORA0125_H.A20170101.0200.002.nc
NLDAS_FORA0125_H.A20170101.0300.002.nc
NLDAS_FORA0125_H.A20170101.0400.002.nc
NLDAS_FORA0125_H.A20170101.0500.002.nc
NLDAS_FORA0125_H.A20170101.0600.002.nc
NLDAS_FORA0125_H.A20170101.0700.002.nc
NLDAS_FORA0125_H.A20170101.0800.002.nc
NLDAS_FORA0125_H.A20170101.0900.002.nc


Let's view the data in the NLDAS2 inputs:

In [8]:
import xarray as xr
import hvplot.xarray

ldas_in = xr.open_mfdataset('/home/docker/wrf-hydro-training/example_case/regridding/NLDAS/input_files/NLDAS_FORA0125_H.A20170101.*.002.nc')

input_field = ldas_in.APCP_surface

input_field.hvplot(groupby='time', data_aspect=1.0, xaxis=None, yaxis=None,
                   title=input_field.short_name, attr_labels=False, widget_location='bottom',
                   cmap='gist_ncar', clim=(input_field.min(), input_field.max()))

Column
    [0] HoloViews(DynamicMap, widget_location='bottom')
    [1] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] DiscreteSlider(margin=(20, 20, 20, 20), name='time', options=OrderedDict([('2017-01-01 ...]), value=numpy.datetime64('2017-01-..., value_throttled=numpy.datetime64('2017-01-..., width=250)
        [2] HSpacer()

**Step 3: View the domain metadata files**

The Forcing Engine requires two additional static input files; a WRF *geogrid* file (for grid information), and an optional Spatial Metadata template file to copy attributes from.

In [10]:
%%bash
ls ~/wrf-hydro-training/example_case/regridding/NLDAS/*nc

/home/docker/wrf-hydro-training/example_case/regridding/NLDAS/geo_em.d0x.nc
/home/docker/wrf-hydro-training/example_case/regridding/NLDAS/GEOGRID_LDASOUT_Spatial_Metadata.nc


We have the necessary files, now we can begin the regridding process.

## Editing the forcing engine configuration file
The WRF-Hydro Forcing Engine requires a configuration file to specify the input types, forecast time periods, and other options needed for regridding and pre-processing the forcing data.

**Step 1: Examine the supplied *nldas_fe.config* configuration file and verify the times and forcing input types**

In [18]:
%%bash
cat ~/wrf-hydro-training/example_case/regridding/NLDAS/nldas_fe.config

#--------------------------------------------------------------------
# WRF-Hydro Forcing Engine Configuration File
#
# Input options to the forcing engine include:
# 1.) Choices for input forcing files to use.
# 2.) Options for specifying date ranges and forecast intervals
#     for input files.
# 3.) Choices for ESMF regridding techniques.
# 4.) Choices for optional downscaling techniques.
# 5.) Choices for optional bias correction techniques.
# 6.) Choices for optional supplemental precipitation products.
# 7.) Choices for optional ensemble member variations.
# 8.) Choices for output directories to place final output files.

[Input]
# Choose a set of value(s) of forcing variables to be processed for
# WRF-Hydro. Please be advised that the order of which the values are
# chosen below are the order that the final products will be layered
# into the final LDASIN files. See documentation for additional
# information and examples.
# The following is a global set of key values to map forc

## Regridding the forcing data using the forcing engine
In this section we will regrid all our forcing data using the configuration shown above

**Step 1: Run the Forcing Engine driver (`genForcing.py`)**

`genForcing.py` takes 3 arguments: the path to the configuration file, a version number, and a configuration name. The version and name are arbitrary and used for logging and are inserted as netCDF Global Attributes in the LDASIN output files.

In [19]:
%%bash
cd ~/wrf-hydro-training/example_case/regridding/NLDAS
rm -rf output_files/* LOG_NLDAS*          # clean up any existing runs
python ../../../WrfHydroForcing/genForcing.py nldas_fe.config 5.2.0 NLDAS

The Forcing Engine sends most of its output to a log file specified in the configuration file. Only severe errors are printed to *stdout* so a successful run will produce no output.

**Step 2: View the Forcing Engine Log:**

In [21]:
%%bash
cat /home/docker/wrf-hydro-training/example_case/regridding/NLDAS/LOG_NLDAS*

[07/28 21:29:47]: INFO - RANK: 0 - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[07/28 21:29:47]: INFO - RANK: 0 - Processing Forecast Cycle: 2017-01-01 00:00
[07/28 21:29:47]: INFO - RANK: 0 - Forecast Cycle Length is: 1440 minutes
[07/28 21:29:47]: INFO - RANK: 0 - =========================================
[07/28 21:29:47]: INFO - RANK: 0 - Processing for output timestep: 2017-01-01 01:00
[07/28 21:29:47]: INFO - RANK: 0 - RAP file being used: /home/docker/wrf-hydro-training/example_case/regridding/NLDAS/input_files/NLDAS_FORA0125_H.A20170101.0000.002.nc
[07/28 21:29:47]: INFO - RANK: 0 - Processing Conus RAP Variable: TMP
[07/28 21:29:47]: INFO - RANK: 0 - Calculating RAP regridding weights.
[07/28 21:29:47]: INFO - RANK: 0 - Creating weight object from ESMF
[07/28 21:29:48]: INFO - RANK: 0 - Finished generating weight object with ESMF, took 0.7859145170077682 seconds
[07/28 21:29:48]: INFO - RANK: 0 - Regridding RAP surface elevation data to the WRF-Hydro domain.
[07/28 21:29:48]: INFO -

**Step 3: View the contents of the *output_files* directory**

The Forcing Engine will generate output files in a directory heirarchy rooted in `output_files`. Lets view the contents of the directory:

In [22]:
%%bash
ls -R ~/wrf-hydro-training/example_case/regridding/NLDAS/output_files

/home/docker/wrf-hydro-training/example_case/regridding/NLDAS/output_files:
2017010100

/home/docker/wrf-hydro-training/example_case/regridding/NLDAS/output_files/2017010100:
201701010100.LDASIN_DOMAIN1
201701010200.LDASIN_DOMAIN1
201701010300.LDASIN_DOMAIN1
201701010400.LDASIN_DOMAIN1
201701010500.LDASIN_DOMAIN1
201701010600.LDASIN_DOMAIN1
201701010700.LDASIN_DOMAIN1
201701010800.LDASIN_DOMAIN1
201701010900.LDASIN_DOMAIN1
201701011000.LDASIN_DOMAIN1
201701011100.LDASIN_DOMAIN1
201701011200.LDASIN_DOMAIN1
201701011300.LDASIN_DOMAIN1
201701011400.LDASIN_DOMAIN1
201701011500.LDASIN_DOMAIN1
201701011600.LDASIN_DOMAIN1
201701011700.LDASIN_DOMAIN1
201701011800.LDASIN_DOMAIN1
201701011900.LDASIN_DOMAIN1
201701012000.LDASIN_DOMAIN1
201701012100.LDASIN_DOMAIN1
201701012200.LDASIN_DOMAIN1
201701012300.LDASIN_DOMAIN1
201701020000.LDASIN_DOMAIN1
WrfHydroForcing.COMPLETE


**Step 4: View the contents of the LDASIN files**

Now, let's look at the actual regridded data in the output files. Try changing the variable from "RAINRATE" to others in the file to see the other fields.

In [23]:
import xarray as xr
import hvplot.xarray

ldas_in = xr.open_mfdataset('/home/docker/wrf-hydro-training/example_case/regridding/NLDAS/output_files/2017010100/2017010*.LDASIN_DOMAIN1')

ldas_in

<xarray.Dataset>
Dimensions:         (reference_time: 1, time: 24, x: 15, y: 16)
Coordinates:
  * time            (time) datetime64[ns] 2017-01-01T01:00:00 ... 2017-01-02
  * reference_time  (reference_time) datetime64[ns] 2017-01-01
  * x               (x) float64 1.843e+06 1.844e+06 ... 1.856e+06 1.857e+06
  * y               (y) float64 4.215e+05 4.225e+05 ... 4.355e+05 4.365e+05
Data variables:
    crs             (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    U2D             (time, y, x) float32 dask.array<chunksize=(1, 16, 15), meta=np.ndarray>
    V2D             (time, y, x) float32 dask.array<chunksize=(1, 16, 15), meta=np.ndarray>
    LWDOWN          (time, y, x) float32 dask.array<chunksize=(1, 16, 15), meta=np.ndarray>
    RAINRATE        (time, y, x) float32 dask.array<chunksize=(1, 16, 15), meta=np.ndarray>
    T2D             (time, y, x) float32 dask.array<chunksize=(1, 16, 15), meta=np.ndarray>
    Q2D             (time, y, x) float32 dask.array<chunksize=(1, 16, 15), meta=np.ndarray>
    PSFC            (time, y, x) float32 dask.array<chunksize=(1, 16, 15), meta=np.ndarray>
    SWDOWN          (time, y, x) float32 dask.array<chunksize=(1, 16, 15), meta=np.ndarray>
Attributes:
    model_output_valid_time:    2017-01-01_01:00:00
    model_initialization_time:  2017-01-01_00:00:00
    NWM_version_number:         v5.2.0
    model_configuration:        NLDAS
    model_output_type:          forcing
    model_total_valid_times:    24
    Conventions:                CF-1.5
    GDAL_DataType:              Generic
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    region:                     CONUS
    NCO:                        netCDF Operators version 4.7.9 (Homepage = ht...

In [24]:
output_field = ldas_in.RAINRATE
output_field.hvplot(groupby='time', data_aspect=1.0, xaxis=None, yaxis=None, 
                    title=output_field.name, attr_labels=False, widget_location='bottom',
                    cmap='gist_ncar', clim=(output_field.min(), output_field.max()))

Column
    [0] HoloViews(DynamicMap, widget_location='bottom')
    [1] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] DiscreteSlider(margin=(20, 20, 20, 20), name='time', options=OrderedDict([('2017-01-01 ...]), value=numpy.datetime64('2017-01-..., value_throttled=numpy.datetime64('2017-01-..., width=250)
        [2] HSpacer()

© UCAR 2020